In [2]:
import nltk
from nltk.corpus import brown
import numpy as np
from collections import Counter

In [3]:
'''
Contains useful utility functions use throughout other areas in our code.
'''

class MyCounter(Counter):
    """
    A myCounter keeps track of counts for a set of keys.

    The myCounter class is an extension of the the standard python Counter class.

    The myCounter also includes additional functionality useful in implementing
    the vectorization of sentences. In particular, counters can be normalized,
    multiplied, etc.
    """
    def __getitem__(self, idx):
        self.setdefault(idx, 0)
        return dict.__getitem__(self, idx)

    def incrementAll(self, keys, count):
        """
        Increments all elements of keys by the same count.

        >>> a = Counter()
        >>> a.incrementAll(['one','two', 'three'], 1)
        >>> a['one']
        1
        >>> a['two']
        1
        """
        for key in keys:
            self[key] += count

    def argMax(self):
        """
        Returns the key with the highest value.
        """
        if len(self.keys()) == 0: return None
        all = self.items()
        values = [x[1] for x in all]
        maxIndex = values.index(max(values))
        return all[maxIndex][0]

    def sortedKeys(self):
        """
        Returns a list of keys sorted by their values.  Keys
        with the highest values will appear first.

        >>> a = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> a['third'] = 1
        >>> a.sortedKeys()
        ['second', 'third', 'first']
        """
        sortedItems = self.items()
        compare = lambda x, y:  sign(y[1] - x[1])
        sortedItems.sort(cmp=compare)
        return [x[0] for x in sortedItems]

    def totalCount(self):
        """
        Returns the sum of counts for all keys.
        """
        return sum(self.values())

    def normalize(self):
        """
        Edits the counter such that the total count of all
        keys sums to 1.  The ratio of counts for all keys
        will remain the same. Note that normalizing an empty
        Counter will result in an error.
        """
        total = float(self.totalCount())
        if total == 0: return
        for key in self.keys():
            self[key] = self[key] / total

    def divideAll(self, divisor):
        """
        Divides all counts by divisor
        """
        divisor = float(divisor)
        for key in self:
            self[key] /= divisor

    def copy(self):
        """
        Returns a copy of the counter
        """
        return MyCounter(dict.copy(self))

    def __mul__(self, y ):
        """
        Multiplying two counters gives the dot product of their vectors where
        each unique label is a vector element.

        >>> a = Counter()
        >>> b = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> b['first'] = 3
        >>> b['second'] = 5
        >>> a['third'] = 1.5
        >>> a['fourth'] = 2.5
        >>> a * b
        14
        """
        sum = 0
        x = self
        if len(x) > len(y):
            x,y = y,x
        for key in x:
            if key not in y:
                continue
            sum += x[key] * y[key]
        return sum


In [90]:
def stationary(Mat, epsilon=0.0001):
    '''
    Given numpy matrix Mat, returns the vector s such that sX = s, where s is normalized 
    to be a probabiliy distribution.
    So we have sX = sI -> s(X-I) = 0, so we need to find ker(X-I).
    We use the linealg package in numpty to take care of this for us. 
    '''
    values, vectors = np.linalg.eig(Mat.T)
    
    # Due to floating point imprecision, need to use epsilon values!
    index = np.nonzero(abs(values - 1.0) < epsilon)
    q = vectors[:,index]
    
    return q / np.sum(q)  # convert into probability distribution
    

In [131]:
def invertMatrixTheorem(A, Ainv, indx):
    '''
    Computes the inverse of a matrix with one row and column removed using
    the matrix inversion lemma. It needs a matrix A, the inverse of A,
    and the row and column index which needs to be removed.
    '''
    n,m = A.shape
    assert(n == m)  # square matrix
    
    # Remove row and compute inverse
    u = np.zeros(n)
    u[indx] = -1
    
    v = A[indx, :]
    
    T1 = v.dot(Ainv)
    T1.shape = (1, n)
    T2 = Ainv.dot(u.T)
    T2.shape = (n, 1)
    T = Ainv - T2.dot(T1) / (1 + T1.dot(u.T))
    
    # Remove column and compute inverse. 
    w = A[:, indx]
    w.shape = n
    w[indx] = 0
    
    R1 = T.dot(w)
    R1.shape = (n,1)
    R2 = u.T.dot(T)
    R2.shape = (1,n)
    
    R = T - R1.dot(R2) / (1 + R2.dot(w))
    
    # Remove redundant rows
    R = np.delete(R, (indx), axis=0)
    R = np.delete(R, (indx), axis=1)
    
    return R

In [174]:
'''
Grasshopper: Python implementation of re-ranking algorithm with absobing states.

For a Matlab Implementation, see http://pages.cs.wisc.edu/~jerryzhu/pub/grasshopper.m
'''

def grasshopper(W, r, lamb, k, epsilon=0.0001):
    '''
    INPUTS:
        W: Matrix representataion of an directed, weighed graph. Self-edges are allowed.
            Undirected graphs should be symetric.
        r: A probability distribution over the sentences, with higher probability implying higher 
            ranking.
        lamb: How much to weight the prior in comparison to the input graph weights.
        k: return the top k items as ranked by Grasshopper
        
    OUTPUT
        res: a list of (index, prob) of length k, where index corresponds to the index in the 
        input graph, and prob is the probability assigned to it when selected.
        
    The algorithm is modified to take advantage of the fact that we only care about the expected
    number of states. 
    '''
    # Let's do some basis error checking!
    n,m = W.shape
    assert(n == m) # Sizes should be equal
    assert(np.min(W) >= 0) # No negative edges
    assert(abs(np.sum(r)- 1) < epsilon) # r is a distribution, floating point imprecision
    assert(0 <= lamb and lamb <= 1) # lambda is valid
    assert(0 < k and k <= n) # Summary can't be longer than document!

    # Normalize the rows of W to create the transition matrix P'
    P = W / np.sum(W, axis=1)
    hatP = lamb * P - (1 - lamb) * r
    
    assert(hatP.shape == (n,m)) #  Shape should not change!
    
    # To store results.
    absorbed = []
    nonAbsorbed = range(n)
    probs = []
    
    # Calculate the most probable state!
    q = stationary(hatP);
    absorbed.append(np.argmax(q))
    probs.append(np.max(q))
    nonAbsorbed.remove(np.argmax(q))
    
    # Compute the inverse of the fundamental matrix!
    N = np.linalg.inv(np.identity(len(nonAbsorbed)) - hatP[nonAbsorbed, nonAbsorbed])
    
    # Pick the ramaining k-1 items by picking out the most-visited node one by one.
    # once picked out, the item turns into an absorbing node.
    while (len(absorbed)<k):
        # Compute expected number fo times each node will be visited before random walk is absorbed by
        # absorbing nodes. Averaged over all start nodes.
    
        # Compute the expected visit counts
        nuvisit = np.sum(N, axis=0)
        nvisit = np.zeros(n)
        nvisit[nonAbsorbed] = nuvisit

        # Find the new absorbing state
        absorbState = np.argmax(nvisit)
        absorbVisit = max(nvisit)
        # Store the results
        absorbed.append(absorbState)
        probs.append(absorbVisit)
        
        # Compute the inverse using the matrix inversion theorem to avoid re-computation!
        index = np.nonzero(abs(nonAbsorbed - absorbState) < epsilon)  # Compute the index of the absorbedState in relation to the submatrix Q (see paper for details)
        N = invertMatrixTheorem(np.identity(len(nonAbsorbed)) - hatP[nonAbsorbed, nonAbsorbed], N, index)
        
        # Update the nonAbsorbed states
        nonAbsorbed.remove(absorbState)
        
    # Return the results!
    return zip(absorbed, probs)        

In [175]:
def clean(w):
    '''
    Given a word, removes all non-alphabetic starting/ending characters. It then proceeds to check 
    if remaining characters are alphabetic. If so, return those characters. If not, returns None
    '''
    # Use regex!
    return w

def cosineSim(v1, v2):
    '''
    Note that input vectors are sparse!
    '''
    nv1 = v1.copy()
    nv2 = v2.copy()
    nv1.normalize()
    nv2.normalize()
    return nv1 * nv2

def thresholdCosineSim(v1,v2, threshold = 0.01):
    score = cosineSim(v1,v2)
    return 0 if score < threshold else score
    
def tf_idf(sentence):
    '''
    Given a sentence, converts the sentence to a TF-IDF representations. The representation is parse,
    with the key being the term. 
    '''
    v1 = MyCounter()
    # TODO(nautilik): Need to avoid stop words, non-english words, etc.
    for word in sentence:
        # Remove starting/ending punctuations/spaces
        # Make sure whatever is left is an english word
        cleanWord = clean(word)
        if cleanWord is not None:
            v1[cleanWord.lower()] +=1
        
    return v1

def docToMatrix(D, vec_fun=tf_idf, sim_fun=cosineSim):
    '''
    Given a document d which consists of a set of sentences, converts it into a |D|_s x |D|_s matrix 
    with weights given by the specifiend similarity function. The similary function should take 
    as input vector representations as output by the vec_fun. 
    '''
    # Convert sentences to vector representations!
    sentenceVectors = [vec_fun(s) for s in D]
    
    # Compute similaliry
    n = len(D)
    M = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            M[i,j] = sim_fun(sentenceVectors[i], sentenceVectors[j])
    return M

In [176]:
W = docToMatrix(brown.sents('ca19'), sim_fun=thresholdCosineSim)

In [190]:
r = np.ones(len(W))
r = r / np.sum(r)
results = grasshopper(W, r, 1.0, 5)

In [191]:
results

[(32, 0.0088495575221240533),
 (70, 0.78209433481689405),
 (2, 0.76883969341860658),
 (30, 0.7405060383741493),
 (10, 0.73698511909308306)]

In [192]:
for (i, p) in sorted(results, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

Howard E. Simpson , the railroad's president , said , `` A drastic decline in freight loading due principally to the severe slump in the movement of heavy goods has necessitated this regrettable action '' .
Since the railroad cannot reduce the salary of individual union members under contract , it must accomplish its payroll reduction by placing some of the men on furlough , a B. & O. spokesman said .
The proposal was made by Dr. David S. Jenkins after he and Mrs. D. Ellwood Williams , Jr. , a board member and long-time critic of the superintendent , argued for about fifteen minutes at this week's meeting .
Cites discrepancies
Soon after 10 A.M. , when police reached the 1-1/2-story brick home in the Franklin Manor section , 15 miles south of here on the bay , in response to a call from the Dresbach's other son , Lee , 14 , they found Mrs. Dresbach's body on the first-floor bedroom floor .
